# Solution for Lilly Technical Challenge

### Imports

In [34]:
import pandas as pd
import numpy as np

### Objective 1

In [35]:
# Import data and convert date columns to date objects
resultsTable = pd.read_csv("results.csv")
resultsTable['date'] = pd.to_datetime(resultsTable['date'])

# Extract year to new column
resultsTable['year'] = resultsTable['date'].dt.year

# Get rows with a date between 1900 and 2000 and get only necessary columns
filtered_results = resultsTable[(resultsTable['year'] >= 1900) & (resultsTable['year'] <= 2000)][['year', 'home_score', 'away_score']]

# Calculate total score
filtered_results['total_goals'] = filtered_results['home_score'] + filtered_results['away_score']

# Get mean for column 'total_goals'
scoreMean = filtered_results['total_goals'].mean()

print(f'Average goals scored between the years 1900 and 2000 is {scoreMean:.2f}')

Average goals scored between the years 1900 and 2000 is 3.07


### Objective 2

In [36]:
shootoutsTable = pd.read_csv("shootouts.csv")
winner_counts = shootoutsTable["winner"].value_counts().sort_index()
print(winner_counts)

winner
Abkhazia                2
Algeria                 7
Angola                  7
Antigua and Barbuda     2
Argentina              14
                       ..
Zambia                 13
Zanzibar                2
Zimbabwe                7
Åland                   1
Åland Islands           2
Name: count, Length: 163, dtype: int64


### Objective 3

In [92]:

# Import data and convert date column to dt object
goalScorersTable = pd.read_csv("goalscorers.csv")
goalScorersTable['date'] = pd.to_datetime(goalScorersTable['date'])
resultsTable = pd.read_csv("results.csv")
resultsTable['date'] = pd.to_datetime(resultsTable['date'])
shootoutsTable = pd.read_csv("shootouts.csv")
shootoutsTable['date'] = pd.to_datetime(shootoutsTable['date'])

# Create a match_id key to merge the tables by 
goalScorersTable["match_id"] = (
    goalScorersTable["date"].astype(str) + "_" +
    goalScorersTable["home_team"].str.replace(" ", "_") + "_" + 
    goalScorersTable["away_team"].str.replace(" ", "_")
    )
resultsTable["match_id"] = (
    resultsTable["date"].astype(str) + "_" +
    resultsTable["home_team"].str.replace(" ", "_") + "_" + 
    resultsTable["away_team"].str.replace(" ", "_")
    )
shootoutsTable["match_id"] = (
    shootoutsTable["date"].astype(str) + "_" +
    shootoutsTable["home_team"].str.replace(" ", "_") + "_" + 
    shootoutsTable["away_team"].str.replace(" ", "_")
    )

# Create goal_id in goalScorersTable
goalScorersTable["goal_id"] = (
    goalScorersTable["match_id"] + "_" +
    goalScorersTable["minute"].astype(str)
)

print(resultsTable.head())

        date home_team away_team  home_score  away_score tournament     city  \
0 1872-11-30  Scotland   England           0           0   Friendly  Glasgow   
1 1873-03-08   England  Scotland           4           2   Friendly   London   
2 1874-03-07  Scotland   England           2           1   Friendly  Glasgow   
3 1875-03-06   England  Scotland           2           2   Friendly   London   
4 1876-03-04  Scotland   England           3           0   Friendly  Glasgow   

    country  neutral                     match_id  
0  Scotland    False  1872-11-30_Scotland_England  
1   England    False  1873-03-08_England_Scotland  
2  Scotland    False  1874-03-07_Scotland_England  
3   England    False  1875-03-06_England_Scotland  
4  Scotland    False  1876-03-04_Scotland_England  


### Objective 4

In [ ]:
# Merge shootouts table with results on previously created 'match_id'
mergedTable = shootoutsTable.merge(resultsTable, on="match_id", how="inner")

# Keep rows which the game results in a score of 1-1
draw_1_1 = mergedTable[(mergedTable["home_score"] == 1) & (mergedTable["away_score"] == 1)]

# Get value counts for winners
shootout_winners_count = draw_1_1["winner"].value_counts()

print(shootout_winners_count)

winner
South Korea            7
Uruguay                7
Burkina Faso           6
Argentina              6
Mali                   5
                      ..
Antigua and Barbuda    1
Zambia                 1
Cuba                   1
Sudan                  1
Åland                  1
Name: count, Length: 115, dtype: int64


### Objective 5

In [99]:

# Merge the appropriate tables
mergedTable = goalScorersTable.merge(resultsTable, on="match_id", how="inner")

# Group by tournament and scorer, get the number of entried for each group, reset the index
goal_counts = mergedTable.groupby(['tournament', 'scorer']).size().reset_index(name='goals')

# For each tournament get the highest scorers index, get rows from original table with index
top_scorer = goal_counts.loc[goal_counts.groupby('tournament')['goals'].idxmax()]

# Get the total goals by tournament and reset the index
total_goals_per_tournament = goal_counts.groupby('tournament')['goals'].sum().reset_index(name='total_goals')

# Merge the new tables by the matching tournament
top_scorer = top_scorer.merge(total_goals_per_tournament, on="tournament")

# Create new column for percentage of tournament goals
top_scorer['goal_percentage'] = (top_scorer['goals'] / top_scorer['total_goals']) * 100

# Display the result
print(top_scorer[['tournament', 'scorer', 'goals', 'total_goals', 'goal_percentage']])


                        tournament                   scorer  goals  \
0                    AFC Asian Cup                 Ali Daei     14   
1           African Cup of Nations             Samuel Eto'o     18   
2                       Baltic Cup          Ēriks Pētersons      9   
3        British Home Championship              Geoff Hurst      4   
4   CONMEBOL–UEFA Cup of Champions         Claudio Caniggia      1   
5               Confederations Cup        Cuauhtémoc Blanco      9   
6                     Copa América  Norberto Doroteo Méndez     17   
7                   FIFA World Cup           Miroslav Klose     16   
8     FIFA World Cup qualification              Carlos Ruiz     39   
9                         Gold Cup           Landon Donovan     18   
10               NAFC Championship        Luis de la Fuente      3   
11             Oceania Nations Cup              Damian Mori     14   
12                       UEFA Euro        Cristiano Ronaldo     14   
13         UEFA Euro

### Additional Objectives

Ran out of the 1 hour time to complete, however I did complete some checks whilst working through the other objectives like checking for Null values.

Additionally to check for data quality issues I would first flag entries with Null values, inconsistent values (such as negative goals), outliers (more than a 100 goals by a player in a tournament), Dulplicates, wrong data type (Such as a number being passed as a player name).

To correct these issues, the most likely option would be remove entries with quality issues. But in some cases it might be possible to correct the issues such as replacing a value with the mean of that column, or replacing Null values with a keyword like "Unknown".